In [1]:
from typing import Any
from bloqade import squin
from kirin.dialects.ilist import IList
import numpy as np
import bloqade.stim
import bloqade.tsim
from bloqade.types import Qubit, MeasurementResult
Register = IList[Qubit, Any]
from bloqade.pyqrack import StackMemorySimulator

In [4]:
def ghz_constant_depth(n_qubits: int):

    @squin.kernel
    def main() -> Register:
        qreg = squin.qalloc(n_qubits)
        ancilla = squin.qalloc(n_qubits - 1)

        for i in range(n_qubits):
            squin.h(qreg[i])

        for i in range(n_qubits - 1):
            squin.cx(qreg[i], ancilla[i])
        for i in range(n_qubits - 1):
            squin.cx(qreg[i + 1], ancilla[i])

        parity: int = 0
        bits = squin.broadcast.measure(ancilla)
        for i in range(n_qubits - 1):
            parity = parity ^ bits[i]
            if parity == 1:
                squin.x(qreg[i + 1])
        return qreg

    return main

In [5]:
emulator = StackMemorySimulator(min_qubits=7)
task = emulator.task(ghz_constant_depth(3))

state = task.batch_state(shots=1000, qubit_map=lambda x: x)
# Even though there is measurement and feedforward, the final state is still pure. Neat!
print(state.eigenvalues)

[1.00000007]
